_READ CAREFULLY_

So, many things to consider now. I did just some brief tests, and first of all, Random Forests do not accept NaN value, so we have to choose the best way to deal with that on an attribute to attribute basis. Also remember that all the columns need to have the same number of rows.
Second, by default it doesnt like strings, which we have in every column expect a few, so we need to check for that. I am pretty confident that we just need to update a few parameters or smth, but now I am tired.
(btw: right now the forest is taking as input 2 integer columns and as label a float columns, those are the only numbers we have)

In [16]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

## Splitting the data into training and test data, maybe also validation

In [17]:
raw_dataset = pd.read_csv('../data/okcupid.csv') 
okcupid_profiles = raw_dataset.drop(columns="Unnamed: 0") 

In [18]:
okcupid_profiles.columns

Index(['age', 'status', 'sex', 'orientation', 'body_type', 'diet', 'drinks',
       'drugs', 'education', 'ethnicity', 'height', 'income', 'job',
       'last_online', 'location', 'offspring', 'pets', 'religion', 'sign',
       'smokes', 'speaks'],
      dtype='object')

In [19]:
okcupid_profiles.dtypes

age              int64
status          object
sex             object
orientation     object
body_type       object
diet            object
drinks          object
drugs           object
education       object
ethnicity       object
height         float64
income           int64
job             object
last_online     object
location        object
offspring       object
pets            object
religion        object
sign            object
smokes          object
speaks          object
dtype: object

In [20]:
okcupid_profiles['status'].unique()

array(['single', 'available', 'seeing someone', 'married', 'unknown'],
      dtype=object)

# Random Forest

In [38]:
# X = okcupid_profiles.drop(['income'], axis = 1)
# y = okcupid_profiles['income']

X = okcupid_profiles.loc[:, ['age', 'height']]
X = X.fillna(value = 0)
y = okcupid_profiles['income']
y = y.fillna(value = 0)

# test_size = 0.3   means 70% training set | 30% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [39]:
# n_estimators is the number of trees in the forest
rfc = RandomForestClassifier(n_estimators = 50)
rfc.fit(X_train, y_train)
rfc_prediction = rfc.predict(X_test)

In [40]:
print("Random Forest Classification report")
print(classification_report(y_test, rfc_prediction))
print("Random Forest Confusion Matrix")
print(confusion_matrix(y_test, rfc_prediction))

Random Forest Classification report
              precision    recall  f1-score   support

          -1       0.81      1.00      0.89     14535
       20000       0.23      0.00      0.01       907
       30000       0.00      0.00      0.00       309
       40000       0.00      0.00      0.00       323
       50000       0.00      0.00      0.00       283
       60000       0.00      0.00      0.00       202
       70000       0.00      0.00      0.00       224
       80000       0.00      0.00      0.00       319
      100000       0.11      0.00      0.00       454
      150000       0.00      0.00      0.00       211
      250000       0.00      0.00      0.00        43
      500000       0.00      0.00      0.00        15
     1000000       0.00      0.00      0.00       159

    accuracy                           0.81     17984
   macro avg       0.09      0.08      0.07     17984
weighted avg       0.67      0.81      0.72     17984

Random Forest Confusion Matrix
[[14510     

C:\Users\matte\Documents\python_projects\HR_ML_Project1\proj1VENV\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\matte\Documents\python_projects\HR_ML_Project1\proj1VENV\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\matte\Documents\python_projects\HR_ML_Project1\proj1VENV\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control th